# (CTC-17) Projeto de Buscas
---

## Dependências

In [4]:
import pandas as pd
import heapq as heap
from math import sqrt
from tqdm import tqdm

## Importando os dados

In [6]:
# Import
australia = pd.read_csv('australia.csv', sep = ",")
# Cleaning
australia = australia[['id', 'city', 'lat', 'lng']]
# Printing
australia

,id,city,lat,lng
0,1,Adelaide,-34.928661,138.598633
1,2,Adelaide River,-13.239430,131.107330
2,3,Albany,-35.003101,117.865952
3,4,Albury,-36.074823,146.924006
4,5,Alice Springs,-23.697479,133.883621
...,...,...,...,...
214,215,Woomera,-31.199810,136.832581
215,216,Yamba,-29.435450,153.360306
216,217,Yeppoon,-23.126829,150.744064
217,218,Young,-34.313499,148.301071


## Pré-Game

### Estrutura de Dados

In [8]:
class City:
    def __init__(self, id, name, lat, lng):
        self.id = id
        self.name = name
        self.lat = lat
        self.lng = lng
        self.roads = set()
        self.weight = None

    def successors(self):
        for road in self.roads:
            yield road.city, road.length
    
        
class Road:
    def __init__(self, city, length):
        self.city = city
        self.length = length
        
def add_road_between(city1, city2):
    distance = 1.1 * sqrt((city1.lat - city2.lat)**2 + (city1.lng - city2.lng)**2)
    # city1 -> city2
    road1 = Road(city2, distance)
    city1.roads.add(road1)
    # city2 -> city1
    road2 = Road(city1, distance)
    city2.roads.add(road2)

### Preenchendo a estrutura de dados

In [10]:
cities = [None] # None é só para preencher o id = 0 (outra alternativa seria trabalhar com `índice - 1`)

# Cities
for index, row in australia.iterrows():
    city = City(row['id'], row['city'], row['lat'], row['lng'])
    cities.append(city)
    
# Roads
for city in cities:
    if city:
        if city.id % 2 == 0:
            # Road to (x - 1)
            add_road_between(city, cities[city.id - 1])
            
            # Road to (x + 2) if it exists
            if city.id + 2 < len(cities):
                add_road_between(city, cities[city.id + 2])
                
        elif city.id > 2:
            # Road to (x - 2)
            add_road_between(city, cities[city.id - 2])
            
            # Road to (x + 1) if it exists
            if city.id + 1 < len(cities):
                add_road_between(city, cities[city.id + 1])

## Algoritmo de Busca

### Parâmetros

In [12]:
# Escolher partida e destino (pode colocar o nome da cidade (formato string) ou o id (formato int))
source = 'Alice Springs'
destiny = 'Yulara'

# Também funcionaria
# source = 5
# destiny = 219

### Adiciona informação (Busca com informação)

In [14]:
# Calcula as distâncias das cidades até o destino (em linha reta)

# Encontra o nó da cidade destino
if type(destiny) == str:
    for city in cities:
        if city and city.name == destiny:
            dest_city = city
            break
        dest_city = None
else:
    for city in cities:
        if city and city.id == destiny:
            dest_city = city
            break
        dest_city = None
        

# Encontra o nó da cidade fonte
if type(source) == str:
    for city in cities:
        if city and city.name == source:
            src_city = city
            break
        src_city = None
else:
    for city in cities:
        if city and city.id == source:
            src_city = city
            break
        src_city = None

if dest_city == None or src_city == None:
    print('Cidades não encontradas')


# Para cada distância seta o peso como a distância em linha reta até a cidade destino
for city in cities:
    if city:
        city.weight = sqrt((city.lat - dest_city.lat)**2 + (city.lng - dest_city.lng)**2)

In [16]:
print(src_city.id, src_city.name, len(src_city.roads))

print(dest_city.id, dest_city.name, len(dest_city.roads))

5 Alice Springs 4
219 Yulara 1


### Algoritmo

In [18]:
# Algoritmo A*
class Node:
    def __init__(self, f_value, g_value, element, parent):
        self.f_value = f_value
        self.g_value = g_value
        self.element = element
        self.parent = parent

    def __lt__(self, other):
        if self.f_value != other.f_value:
            return self.f_value < other.f_value
        else:
            return self.element.id < other.element.id

def astar_search(initial_state, goal_test):
    expanded = set()
    # heap de minimo
    pq = [Node(initial_state.weight, 0, initial_state, None)]
    heap.heapify(pq)
    for _ in tqdm(range(1000000)):
        # não há caminho
        if len(pq) == 0:
            return None

        curr = heap.heappop(pq)
        if curr.element in expanded:
            continue
        else:
            expanded.add(curr.element)

        # encontrou o objetivo
        if goal_test(curr.element):
            return curr

        # expandindo vizinhos
        for succ, price in curr.element.successors():
            if succ in expanded:
                continue
            new_g_value = (curr.g_value + price)
            new_f_value = new_g_value + succ.weight
            heap.heappush(pq, Node(new_f_value, new_g_value, succ, curr))

In [ ]:
def goal_test(city):
    return city == dest_city

solution_node = astar_search(src_city, goal_test)

## Resultado

In [17]:
path = []
aux_node = solution_node
while aux_node is not None:
    path.append(aux_node.element)
    aux_node = aux_node.parent
    
path.reverse()
print('LENGTH: ', len(path))
for city in path:
    print(city.name, ' ->')

LENGTH:  124
Alice Springs  ->
Andamooka  ->
Armidale  ->
Ayr  ->
Ballarat  ->
Bairnsdale East  ->
Ballina  ->
Batemans Bay  ->
Bathurst  ->
Bendigo  ->
Bicheno  ->
Birdsville  ->
Bordertown  ->
Bourke  ->
Brisbane  ->
Broome  ->
Bundaberg  ->
Burnie  ->
Byron Bay  ->
Cairns  ->
Caboolture  ->
Caloundra  ->
Canberra  ->
Ceduna  ->
Charleville  ->
Clare  ->
Cobram  ->
Colac  ->
Cowell  ->
Cranbourne  ->
Dalby  ->
Deniliquin  ->
Dubbo  ->
East Maitland  ->
Eidsvold  ->
Esperance  ->
Forbes  ->
Gawler  ->
Georgetown  ->
Gingin  ->
Geraldton  ->
Gladstone  ->
Goondiwindi  ->
Griffith  ->
Gympie South  ->
Hamilton  ->
Hobart  ->
Hughenden  ->
Inverell  ->
Kalbarri  ->
Karratha  ->
Katanning  ->
Katoomba  ->
Kiama  ->
Kimba  ->
Kingoonya  ->
Kingston South East  ->
Kwinana  ->
Laverton  ->
Leonora  ->
Longreach  ->
Manjimup  ->
Maryborough  ->
Meekatharra  ->
Melton  ->
Melbourne  ->
Meningie  ->
Mildura  ->
Morawa  ->
Mount Barker  ->
Mount Isa  ->
Mudgee  ->
Muswellbrook  ->
Narrabri West 